In [197]:
# Libs
import pandas as pd
import os 
import json


In [198]:
pwd = os.getcwd()
# Dataset
df = pd.read_csv(pwd + r"\Data\Data - Aircraft_Incident_Dataset.csv")
dataset = pd.read_csv(pwd + r"\Data\Data - Jadaero Parts Cleaned.csv")
client_dataset = pd.read_csv(pwd + r"\Data\Data - Clients.csv")
dataset = dataset.copy()
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PartNumber   500 non-null    object
 1   PartName     500 non-null    object
 2   Description  495 non-null    object
dtypes: object(3)
memory usage: 11.8+ KB


In [199]:
IC = df['Incident_Cause(es)'].unique().tolist()
IC

# IMPORTING CATEGORIES AND GIVE IT TO DEEPSEEK TO CREATE A DICTIONARY

['Airplane - Engines, Airplane - Engines - Prop/turbine blade separation, Collision - Object, Collision - Object - Bird, Result - Emergency, forced landing - On runway',
 'Airplane - Undercarriage, Airplane - Undercarriage - Landing gear collapse, Result - Runway mishap',
 'Cargo - Fire/smoke, Result - Damaged on the ground',
 'Result - Damaged on the ground',
 'Landing/takeoff - Tailstrike, Result - Runway mishap',
 'Result - Runway excursion',
 'Info-Unavailable',
 'Result - Runway mishap',
 'Result - Loss of control',
 'Result - Loss of control, Result - Loss of control (presumed)',
 'Landing/takeoff - Landing, Landing/takeoff - Landing - Heavy, Result - Runway mishap',
 'Result - Emergency, forced landing - Ditching',
 'Landing/takeoff - Takeoff - Aborted, Result - Runway excursion',
 'Landing/takeoff - Landing, Landing/takeoff - Landing - Wrong runway/taxiway, Result - Runway excursion',
 'Airplane - Engines, Airplane - Engines - All engine powerloss, Result - Emergency, forced la

In [200]:
# Load JSON file
with open(pwd + '\Dics\Dic1 - Airsystems.json', 'r') as file:
    aircraft_systems = json.load(file)

# Match part numbers and descriptions to categories
def categorize_part(part_name, description):
    description = str(description).lower()
    part_name = str(part_name).lower()
    
    for category, data in aircraft_systems.items():
        # Check if part name matches known patterns
        if any(common_part.lower() in part_name for common_part in data['common_parts']):
            return category
        # Check description keywords
        if any(keyword.lower() in description for keyword in data['keywords']):
            return category
        # Check part name keywords too
        if any(keyword.lower() in part_name for keyword in data['keywords']):
            return category
    return 'other'

def get_risk_level(part_name, description):
    # Get risk level for aircraft parts based on categorization"""
    category = categorize_part(part_name, description)
    
    if category in aircraft_systems:
        return aircraft_systems[category]['risk_level']
    else:
        return 'Unknown'

def get_primary_replacement_interval(part_name, description):
    # Get the most relevant replacement interval for a specific part"""
    category = categorize_part(part_name, description)
    part_name_upper = str(part_name).upper()
    
    if category in aircraft_systems:
        replacement_dict = aircraft_systems[category].get('replacement_interval', {})
        
        # Try to find the most specific match first
        for component, interval in replacement_dict.items():
            if any(word in part_name_upper for word in component.upper().split()):
                return interval
        
        # If no specific match, return the first available interval
        if replacement_dict:
            return list(replacement_dict.values())[0]
    
    return 'On-condition'

def get_boeing_targets(part_name, description):
    # Get Boeing targets for the part category"""
    category = categorize_part(part_name, description)
    if category in aircraft_systems:
        return ', '.join(aircraft_systems[category].get('boeing_targets', []))
    return ''

def get_bombardier_targets(part_name, description):
    # Get Bombardier targets for the part category"""
    category = categorize_part(part_name, description)
    if category in aircraft_systems:
        return ', '.join(aircraft_systems[category].get('bombardier_targets', []))
    return ''

def get_mro_targets(part_name, description):
    # Get MRO targets for the part category"""
    category = categorize_part(part_name, description)
    if category in aircraft_systems:
        return ', '.join(aircraft_systems[category].get('mro_targets', []))
    return ''

def get_geo_targets(part_name, description):
    # Get geographical targets for the part category
    category = categorize_part(part_name, description)
    if category in aircraft_systems:
        return ', '.join(aircraft_systems[category].get('geo_targets', []))
    return ''

# Apply to your DataFrame
dataset['DamageCategory'] = dataset.apply(lambda x: categorize_part(x['PartName'], x['Description']), axis=1)
dataset['risk_level'] = dataset.apply(lambda x: get_risk_level(x['PartName'], x['Description']), axis=1)
dataset['replacement_interval'] = dataset.apply(lambda x: get_primary_replacement_interval(x['PartName'], x['Description']), axis=1)
dataset['boeing_targets'] = dataset.apply(lambda x: get_boeing_targets(x['PartName'], x['Description']), axis=1)
dataset['bombardier_targets'] = dataset.apply(lambda x: get_bombardier_targets(x['PartName'], x['Description']), axis=1)
dataset['mro_targets'] = dataset.apply(lambda x: get_mro_targets(x['PartName'], x['Description']), axis=1)
dataset['geo_targets'] = dataset.apply(lambda x: get_geo_targets(x['PartName'], x['Description']), axis=1)

# View results
dataset[dataset['DamageCategory'] != 'other'].sample(10)

,PartNumber,PartName,Description,DamageCategory,risk_level,replacement_interval,boeing_targets,bombardier_targets,mro_targets,geo_targets
179,3214-31,BATTERY PACK,"Part Number: 3214-31\r\rDescription: BATTERY, ...",electrical_systems,High,2 years (or capacity < 80%),"B737 electrical, B777 electrical, B787 electrical","CRJ electrical, Global electrical","Electrical component repair, Wire harness repl...",Older fleets needing upgrades + EM interferenc...
419,8396-5,HORIZONTAL STABILIZER TRIM ACTUATOR,"Part Number: 8396-5\r\rDescription: HSTA, ACTU...",flight_controls,Extreme,"10,000hrs (or current spikes)","B737 FCC, B777 PFCS, B787 FBW","CRJ FCS, Global FBW","Flight control rigging, Actuator overhaul, Com...","High-frequency routes (US domestic, Europe-Mid..."
190,039-853-0,MAIN TIRE,Part Number: 039-853-0\r\rDescription: MICHELI...,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid..."
45,APT-482B-1250-60D,"TRANSDUCER, FUEL FILTER DP",Part Number: APT-482B-1250-60D\r\rDescription:...,fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...
205,342B050000,TEMPERATURE CONTROL,Part Number: 342B050000\r\rDescription: TEMPER...,hydraulic_systems,High,"5,000hrs (efficiency < 80%)","B737 hydraulics, B777 hydraulics, B787 hydraulics","CRJ hydraulics, Global hydraulics","Hydraulic component overhaul, System flushing,...","High-frequency routes (US domestic, Europe-Mid..."
458,BA03604,GENERATOR,Part Number: BA03604\r\rDescription: GENERATOR...,hydraulic_systems,High,"5,000hrs (efficiency < 80%)","B737 hydraulics, B777 hydraulics, B787 hydraulics","CRJ hydraulics, Global hydraulics","Hydraulic component overhaul, System flushing,...","High-frequency routes (US domestic, Europe-Mid..."
193,040-800-1,MAIN TIRE ASSY,Part Number: 040-800-1\r\rDescription: TIRE: H...,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid..."
171,745-2010-503,FRONT ENGINE MOUNT,Part Number: 745-2010-503\r\rDescription: FORW...,engines,Extreme,On-condition (ultrasound every 500 cycles),"CFM56, GE90, Trent, PW4000, LEAP","PW100, CF34, BR700","Engine overhaul, Hot section repair, Component...","Harsh environments (desert, coastal) + long-ha..."
130,194792-4,PRECOOLER,Part Number: 194792-4\r\rDescription: BLEED-AI...,pneumatic_systems,High,"6,000hrs (vibration monitoring)","B737 pneumatic, B777 pneumatic, B787 no-bleed","CRJ pneumatic, Global pneumatic","Pneumatic system test, ACM overhaul, Valve repair",High-altitude routes + extreme temperature ope...
65,3031863-001,HYDRAULIC PUMP,Part Number: 3031863-001\r\rDescription: PUMP ...,fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...


In [201]:
dataset = dataset.rename(columns={'damage_category': 'DamageCategory'})
dataset[dataset['DamageCategory'] != 'other'].sample(10)

#RENAME COLUMN DamageCategory

,PartNumber,PartName,Description,DamageCategory,risk_level,replacement_interval,boeing_targets,bombardier_targets,mro_targets,geo_targets
454,1152426-245,APU START CONVERTER,Part Number: 1152426-245\r\rDescription: APU S...,ground_support,Low,"On-condition (8,000hrs inspection)","All Boeing aircraft APU, Ground service equipment","All Bombardier aircraft APU, Ground service eq...","APU maintenance, Ground equipment service, Lin...","High-frequency routes (US domestic, Europe-Mid..."
258,65C19789-7,TRANSMITTER,Part Number: 65C19789-7\r\rDescription: TRANSM...,flight_management,Moderate,10 years (or MTBF),"B737 FMS, B777 FMS, B787 FMS","CRJ FMS, Global FMS","Avionics repair, Navigation calibration, Datab...","High-frequency routes (US domestic, Europe-Mid..."
318,GP422-1102-1,VALVE ASSY,Part Number: GP422-1102-1\r\rDescription: VALV...,fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...
2,M01103-02,TIRE – MAIN,Part Number: M01103-02\r\rDescription: 46x17.0...,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid..."
210,1312A0000-01,VALVE,Part Number: 1312A0000-01\r\rDescription: PACK...,fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...
1,M20101-01,TIRE MAIN,NaN,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid..."
453,2119584-4,"CONTROL UNIT, APU",Part Number: 2119584-4\r\rDescription: ELECTRO...,ground_support,Low,"On-condition (8,000hrs inspection)","All Boeing aircraft APU, Ground service equipment","All Bombardier aircraft APU, Ground service eq...","APU maintenance, Ground equipment service, Lin...","High-frequency routes (US domestic, Europe-Mid..."
312,822-1338-205,ATC TRANSPONDER,Part Number: 822-1338-205\r\rDescription: ATC ...,communication_systems,Moderate,10 years (reply efficiency),"B737 comm, B777 comm, B787 comm","CRJ comm, Global comm","Radio repair, Antenna replacement, Communicati...","High-frequency routes (US domestic, Europe-Mid..."
110,9000408-22,TRANSMITTER,Part Number: 9000408-22\r\rDescription: FLAP P...,flight_management,Moderate,10 years (or MTBF),"B737 FMS, B777 FMS, B787 FMS","CRJ FMS, Global FMS","Avionics repair, Navigation calibration, Datab...","High-frequency routes (US domestic, Europe-Mid..."
13,338-067-014-0,FAN OUTLET GUIDE VANE,Part Number: 338-067-014-0\r\rDescription: VAN...,engines,Extreme,On-condition (ultrasound every 500 cycles),"CFM56, GE90, Trent, PW4000, LEAP","PW100, CF34, BR700","Engine overhaul, Hot section repair, Component...","Harsh environments (desert, coastal) + long-ha..."


In [202]:
dataset[dataset['PartName'] == 'ENGINE NEW 675 HP']


,PartNumber,PartName,Description,DamageCategory,risk_level,replacement_interval,boeing_targets,bombardier_targets,mro_targets,geo_targets
499,PT6A-114A,ENGINE NEW 675 HP,NaN,other,Unknown,On-condition,,,,


In [203]:
dataset['DamageCategory'].value_counts()

# SHOW THE UNIQUE VALUES IN THE DamageCategory COLUMN
#  27 ROWS FROM 500 ROWS
# AFTER CLEANING AND SERCHING FOR OTHER PARTS 
#  253 ROWS FROM 500 ROWS

DamageCategory
other                         274
fuel_systems                   56
engines                        32
flight_controls                21
landing_gear                   20
cabin_systems                  18
flight_management              18
hydraulic_systems              15
electrical_systems             10
cargo_systems                   8
air_conditioning                7
surveillance_systems            6
pneumatic_systems               6
regional_aircraft_specific      3
communication_systems           3
ground_support                  3
Name: count, dtype: int64

In [204]:
dataset['Description'] = dataset['Description'].str.split('Description:', n=1,expand=True)[1]  # split the Description column by '\r\r' and keep the second part
dataset.sample(2)

,PartNumber,PartName,Description,DamageCategory,risk_level,replacement_interval,boeing_targets,bombardier_targets,mro_targets,geo_targets
361,735511A,CONSTANT SPEED DRIVE,CSD - CONSTANT SPEED DRIVE,other,Unknown,On-condition,,,,
351,45040-1,STATOR AND ROTOR ASSY,ALTERNATOR STATOR,other,Unknown,On-condition,,,,


In [205]:
dataset.columns

Index(['PartNumber', 'PartName', 'Description', 'DamageCategory', 'risk_level',
       'replacement_interval', 'boeing_targets', 'bombardier_targets',
       'mro_targets', 'geo_targets'],
      dtype='object')

In [206]:
dataset = dataset.rename(columns={'PartNumber':'Part Number','PartName': 'Part Name', 'Description': 'Description', 'DamageCategory': 'Air System Category', 'risk_level': 'Risk Level',
                                  'replacement_interval': 'Replacement Interval', 'boeing_targets': 'Boeing Targets', 'bombardier_targets': 'Bombardier Targets',
                                  'mro_targets': 'MRO Targets', 'geo_targets': 'Geographical Targets'})

In [207]:
dataset.sample(1)

,Part Number,Part Name,Description,Air System Category,Risk Level,Replacement Interval,Boeing Targets,Bombardier Targets,MRO Targets,Geographical Targets
247,A21010-203,EQUIP TEST P.R.V,VACUUM TANK - PRESS RELIEF VALVE TESTER,hydraulic_systems,High,"5,000hrs (efficiency < 80%)","B737 hydraulics, B777 hydraulics, B787 hydraulics","CRJ hydraulics, Global hydraulics","Hydraulic component overhaul, System flushing,...","High-frequency routes (US domestic, Europe-Mid..."


In [208]:
description_null = dataset[dataset['Description'].isnull()]
description_null

,Part Number,Part Name,Description,Air System Category,Risk Level,Replacement Interval,Boeing Targets,Bombardier Targets,MRO Targets,Geographical Targets
0,021-355-0,"TIRE 11.00-12 10PLY 160 mph, 10 Ply",NaN,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid..."
1,M20101-01,TIRE MAIN,NaN,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid..."
12,1159SCL404-5,BRAKE ASSEMBLY,None,other,Unknown,On-condition,,,,
464,3876046-4,SOLENOID VALVE,NaN,fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...
479,472420-2,FIRE EXTINGUISHER,NaN,other,Unknown,On-condition,,,,
499,PT6A-114A,ENGINE NEW 675 HP,NaN,other,Unknown,On-condition,,,,


In [209]:
dataset['Description'] = dataset['Description'].fillna(dataset['Part Name'])
dataset[dataset['Part Name'] == 'TIRE MAIN']

,Part Number,Part Name,Description,Air System Category,Risk Level,Replacement Interval,Boeing Targets,Bombardier Targets,MRO Targets,Geographical Targets
1,M20101-01,TIRE MAIN,TIRE MAIN,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid..."


In [210]:
geo_keywords = []

for geo_target in dataset['Geographical Targets']:
    geo_target = str(geo_target).lower()
    locations = set()  # Use a set to avoid duplicates
    
    if 'us domestic' in geo_target:
        locations.add('United States')
    if 'europe' in geo_target:
        locations.add('Europe')
    if 'middle east' in geo_target:
        locations.add('Middle East')
    if 'desert' in geo_target:
        # 'Sahara (Africa)', 'Sonoran (USA/Mexico)', 'Gobi (China)', 'Arabian Desert (Middle East)'
        locations.add('Desert Locations')
    if 'coastal' in geo_target:
        # 'Casablanca (Morocco)', 'Cape Town (South Africa)', 'Miami (USA)', 'Sydney (Australia)'
        locations.add('Coastal Cities')
    if 'long-haul' in geo_target:
        # 'Long-haul: Europe to Asia, Morocco to US'
        locations.add('Long-haul')
    if not locations:
        locations.add('Unknown')

   # Convert set to comma-separated string (no brackets)
    geo_keywords.append(list(locations))
    
dataset['Geographical Location'] = geo_keywords


In [211]:
dataset.sample(10)

,Part Number,Part Name,Description,Air System Category,Risk Level,Replacement Interval,Boeing Targets,Bombardier Targets,MRO Targets,Geographical Targets,Geographical Location
244,1816B0000-02,PLENUM,CHAMBER-AIR PLENUM,other,Unknown,On-condition,,,,,[Unknown]
416,285A1010-106,SMYD COMPUTER,STALL MANAGEMENT/YAW DAMPER (SMYD),other,Unknown,On-condition,,,,,[Unknown]
383,0856TT1,"TUBE, PITOT",PITOT STATIC TUBE,other,Unknown,On-condition,,,,,[Unknown]
366,GM168-1700-22,SPOILERON,INBOARD RIGHT SPOILERON,other,Unknown,On-condition,,,,,[Unknown]
318,GP422-1102-1,VALVE ASSY,"VALVE, AFT TANK WATER",fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...,[Long-haul]
410,5145-1-64,PANEL ASSY,"PANEL ASSY, AUDIO CONTROL",other,Unknown,On-condition,,,,,[Unknown]
326,GD467-1802-13,TUBE ASSY,WATER INJECTION TUBE,other,Unknown,On-condition,,,,,[Unknown]
321,GG436-2048-7,RAM AIR VALVE,"VALVE, RAM-AIR",fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...,[Long-haul]
296,446164,DISCHARGE HEAD,DISCHARGE HEAD,other,Unknown,On-condition,,,,,[Unknown]
219,3107-89,ANTENNA,ANTENNA MARKER,flight_management,Moderate,10 years (or MTBF),"B737 FMS, B777 FMS, B787 FMS","CRJ FMS, Global FMS","Avionics repair, Navigation calibration, Datab...","High-frequency routes (US domestic, Europe-Mid...","[Long-haul, United States, Europe, Middle East]"


In [212]:


# Client Type 1: Regional Airlines
# Expanded geo pattern for Regional Airlines
geo1_pattern = r"(?i)high-frequency routes|regional routes|high-cycle operations"

clt1_mask = (
    dataset['Replacement Interval'].isin(['10 years', '15 years', '7 years']) |
    dataset['Geographical Targets'].str.contains('High-frequency routes') |
    dataset['Bombardier Targets'].str.contains('Regional aircraft') |
    dataset['Boeing Targets'].str.contains('Regional aircraft')
)



# Client Type 2: Cargo Airlines
Geo = 'Harsh environments|(desert, coastal)|long-haul operations|Hot climates|cargo temperature-sensitive operations'
clt2_mask = (
        dataset['Replacement Interval'].isin(['5,000hrs']) |
        dataset['Geographical Targets'].str.contains(Geo) |
        dataset['MRO Targets'].str.contains('Heavy maintenance')
)



# Client Type 3: MRO Service Providers
Geo2 = 'Safety-critical universal demand|harsh environments'
clt3_mask = (
        dataset['Replacement Interval'].isin(['6 months', '2,000hrs', '500 cycles']) |
        dataset['Geographical Targets'].str.contains(Geo2) |
        dataset['MRO Targets'].str.contains('Primary focus')
)


# Client Type 4: Legacy Fleet Operators
Geo3 = 'Older fleets needing upgrades|EM interference environments'
clt4_mask = (
        dataset['Geographical Targets'].str.contains(Geo3) |
        dataset['Boeing Targets'].str.contains('Older Boeing models')
)

# Step 1: Create a new column with default value (e.g. 0 for "Unclassified")
dataset['Client Number'] = 0
dataset.loc[clt1_mask, 'Client Number'] = 1
dataset.loc[clt2_mask, 'Client Number'] = 2
dataset.loc[clt3_mask, 'Client Number'] = 3
dataset.loc[clt4_mask, 'Client Number'] = 4

#Use (?i) for inline case-insensitive regex.


C:\Users\The Dev Reda\AppData\Local\Temp\ipykernel_11096\3508876311.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dataset['Geographical Targets'].str.contains(Geo) |


In [213]:
dataset

,Part Number,Part Name,Description,Air System Category,Risk Level,Replacement Interval,Boeing Targets,Bombardier Targets,MRO Targets,Geographical Targets,Geographical Location,Client Number
0,021-355-0,"TIRE 11.00-12 10PLY 160 mph, 10 Ply","TIRE 11.00-12 10PLY 160 mph, 10 Ply",landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...","[Middle East, Coastal Cities, Long-haul, Unite...",3
1,M20101-01,TIRE MAIN,TIRE MAIN,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...","[Middle East, Coastal Cities, Long-haul, Unite...",3
2,M01103-02,TIRE – MAIN,46x17.0R20 30PR 225mph Radial TL Michelin Air…,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...","[Middle East, Coastal Cities, Long-haul, Unite...",3
3,114079019,"VALVE, LANDING GEAR SELECTOR",ELECTRO-HIDRAULIC SELECTORS,landing_gear,Extreme,"15,000 cycles (or structural cracks)","B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...","[Middle East, Coastal Cities, Long-haul, Unite...",3
4,89794077,CYLINDER & VALVE ASSY,PORTABLE OXYGEN CYLINDER AND VALVE ASSEMBLY,fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...,[Long-haul],2
...,...,...,...,...,...,...,...,...,...,...,...,...
495,424634,CARTRIDGE 20PC,CARTRIDGE 20PC PACKAGE KIT 1.4C AS 1.4S,other,Unknown,On-condition,,,,,[Unknown],0
496,G825100-4,FIREX,"FIREX, DISPOSABLE (POTTY)",other,Unknown,On-condition,,,,,[Unknown],0
497,475089-1,FIREX 224 CU,FIREX 224 CU IN DUAL OUTLET,other,Unknown,On-condition,,,,,[Unknown],0
498,474158-3,FIREX,FIREX DUAL OUTLET 1400 CU IN,other,Unknown,On-condition,,,,,[Unknown],0


In [214]:
dataset_merged = dataset.merge(client_dataset, on='Client Number', how='left')
dataset_merged

,Part Number,Part Name,Description,Air System Category,Risk Level,Replacement Interval,Boeing Targets,Bombardier Targets,MRO Targets,Geographical Targets,Geographical Location,Client Number,Client Role
0,021-355-0,"TIRE 11.00-12 10PLY 160 mph, 10 Ply","TIRE 11.00-12 10PLY 160 mph, 10 Ply",landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...","[Middle East, Coastal Cities, Long-haul, Unite...",3,MRO Service Providers
1,M20101-01,TIRE MAIN,TIRE MAIN,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...","[Middle East, Coastal Cities, Long-haul, Unite...",3,MRO Service Providers
2,M01103-02,TIRE – MAIN,46x17.0R20 30PR 225mph Radial TL Michelin Air…,landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...","[Middle East, Coastal Cities, Long-haul, Unite...",3,MRO Service Providers
3,114079019,"VALVE, LANDING GEAR SELECTOR",ELECTRO-HIDRAULIC SELECTORS,landing_gear,Extreme,"15,000 cycles (or structural cracks)","B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...","[Middle East, Coastal Cities, Long-haul, Unite...",3,MRO Service Providers
4,89794077,CYLINDER & VALVE ASSY,PORTABLE OXYGEN CYLINDER AND VALVE ASSEMBLY,fuel_systems,High,"5,000hrs (flow < 90% spec)","B737 fuel system, B777 fuel management, B787 fuel","CRJ fuel, Global fuel","Fuel system maintenance, Tank entry, Pump over...",Hot climates + cargo temperature-sensitive ope...,[Long-haul],2,Cargo Airlines
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,424634,CARTRIDGE 20PC,CARTRIDGE 20PC PACKAGE KIT 1.4C AS 1.4S,other,Unknown,On-condition,,,,,[Unknown],0,Unclassified
496,G825100-4,FIREX,"FIREX, DISPOSABLE (POTTY)",other,Unknown,On-condition,,,,,[Unknown],0,Unclassified
497,475089-1,FIREX 224 CU,FIREX 224 CU IN DUAL OUTLET,other,Unknown,On-condition,,,,,[Unknown],0,Unclassified
498,474158-3,FIREX,FIREX DUAL OUTLET 1400 CU IN,other,Unknown,On-condition,,,,,[Unknown],0,Unclassified


In [215]:
dataset_merged = dataset_merged.explode('Geographical Location')
dataset_merged


,Part Number,Part Name,Description,Air System Category,Risk Level,Replacement Interval,Boeing Targets,Bombardier Targets,MRO Targets,Geographical Targets,Geographical Location,Client Number,Client Role
0,021-355-0,"TIRE 11.00-12 10PLY 160 mph, 10 Ply","TIRE 11.00-12 10PLY 160 mph, 10 Ply",landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...",Middle East,3,MRO Service Providers
0,021-355-0,"TIRE 11.00-12 10PLY 160 mph, 10 Ply","TIRE 11.00-12 10PLY 160 mph, 10 Ply",landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...",Coastal Cities,3,MRO Service Providers
0,021-355-0,"TIRE 11.00-12 10PLY 160 mph, 10 Ply","TIRE 11.00-12 10PLY 160 mph, 10 Ply",landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...",Long-haul,3,MRO Service Providers
0,021-355-0,"TIRE 11.00-12 10PLY 160 mph, 10 Ply","TIRE 11.00-12 10PLY 160 mph, 10 Ply",landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...",United States,3,MRO Service Providers
0,021-355-0,"TIRE 11.00-12 10PLY 160 mph, 10 Ply","TIRE 11.00-12 10PLY 160 mph, 10 Ply",landing_gear,Extreme,200-300 landings (or tread depth < 2mm),"B737, B777, B787, B747, B767","CRJ, DHC-8, Global","Heavy maintenance, Line maintenance, Component...","High-frequency routes (US domestic, Europe-Mid...",Europe,3,MRO Service Providers
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,424634,CARTRIDGE 20PC,CARTRIDGE 20PC PACKAGE KIT 1.4C AS 1.4S,other,Unknown,On-condition,,,,,Unknown,0,Unclassified
496,G825100-4,FIREX,"FIREX, DISPOSABLE (POTTY)",other,Unknown,On-condition,,,,,Unknown,0,Unclassified
497,475089-1,FIREX 224 CU,FIREX 224 CU IN DUAL OUTLET,other,Unknown,On-condition,,,,,Unknown,0,Unclassified
498,474158-3,FIREX,FIREX DUAL OUTLET 1400 CU IN,other,Unknown,On-condition,,,,,Unknown,0,Unclassified


In [216]:
dataset['Geographical Targets'].unique()

array(['High-frequency routes (US domestic, Europe-Middle East), Harsh environments (desert, coastal) + long-haul operations, High-cycle operations + harsh environments',
       'Hot climates + cargo temperature-sensitive operations, High-altitude routes + extreme temperature operations, Long-haul operations',
       '',
       'High-passenger-turnover routes + wear-intensive operations, Long-haul operations + passenger comfort, High-frequency routes (US domestic, Europe-Middle East)',
       'Harsh environments (desert, coastal) + long-haul operations, High-altitude routes + extreme temperature operations, Weather-intensive routes',
       'Harsh environments (desert, coastal) + long-haul operations, High-altitude routes + extreme temperature operations, Safety-critical universal demand + harsh environments',
       'High-frequency routes (US domestic, Europe-Middle East), High-cycle operations + temperature extremes, Harsh environments (desert, coastal) + long-haul operations',
     

In [217]:
dataset_merged['Client Role'].value_counts()

Client Role
MRO Service Providers     288
Unclassified              274
Cargo Airlines            231
Legacy Fleet Operators     94
Regional Airlines          72
Name: count, dtype: int64

In [218]:
dataset_merged[dataset_merged['Replacement Interval'] == '12 years (rubber degradation)']

,Part Number,Part Name,Description,Air System Category,Risk Level,Replacement Interval,Boeing Targets,Bombardier Targets,MRO Targets,Geographical Targets,Geographical Location,Client Number,Client Role
395,MF10-08-01,OXYGEN MASK,"OXYGEN MASK, CREW",cabin_systems,Low,12 years (rubber degradation),"B737 cabin, B777 cabin, B787 cabin","CRJ cabin, Global cabin","Cabin refurbishment, IFE upgrade, Seat repair",High-passenger-turnover routes + wear-intensiv...,Long-haul,1,Regional Airlines
395,MF10-08-01,OXYGEN MASK,"OXYGEN MASK, CREW",cabin_systems,Low,12 years (rubber degradation),"B737 cabin, B777 cabin, B787 cabin","CRJ cabin, Global cabin","Cabin refurbishment, IFE upgrade, Seat repair",High-passenger-turnover routes + wear-intensiv...,United States,1,Regional Airlines
395,MF10-08-01,OXYGEN MASK,"OXYGEN MASK, CREW",cabin_systems,Low,12 years (rubber degradation),"B737 cabin, B777 cabin, B787 cabin","CRJ cabin, Global cabin","Cabin refurbishment, IFE upgrade, Seat repair",High-passenger-turnover routes + wear-intensiv...,Europe,1,Regional Airlines
395,MF10-08-01,OXYGEN MASK,"OXYGEN MASK, CREW",cabin_systems,Low,12 years (rubber degradation),"B737 cabin, B777 cabin, B787 cabin","CRJ cabin, Global cabin","Cabin refurbishment, IFE upgrade, Seat repair",High-passenger-turnover routes + wear-intensiv...,Middle East,1,Regional Airlines


In [219]:
# print(dataset_merged['Geographical Location'].unique()) #Unknown
# print(dataset_merged['Risk Level'].unique()) #Unknown
# print(dataset_merged['Air System Category'].unique()) #other
# print(dataset_merged['Replacement Interval'].unique()) # On-condition
# print(dataset_merged['Client Role'].unique())  #'Unclassified

dataset_filtred = dataset_merged[
    (dataset_merged['Geographical Location'] != 'Unknown') &
    (dataset_merged['Risk Level'] != 'Unknown') & 
    (dataset_merged['Air System Category'] != 'other') &
    (dataset_merged['Replacement Interval'] != 'On-condition') &
    (dataset_merged['Client Role'] != 'Unclassified')
]

In [220]:
dataset_filtred.to_csv(pwd + r"\Data\Data - Jadaero Parts Edited.csv", index=False)
# Save the categorized DataFrame to a new CSV file